In [8]:
#https://query2.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAPL?symbol=AAPL"
#https://query2.finance.yahoo.com/v8/finance/chart/AAPL?period1=1704145844&period2=1723236317&interval=1mo

In [9]:
import sys
import os
import pandas as pd

cwd = os.getcwd()

sys.path.append(cwd.replace('notebooks','')+'DeltaStock')

In [10]:
from source.scraper import get_companyfacts
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json
facts = get_companyfacts('CIK0001045810') #2016-07-31
facts.head()#

,finType,label,description,end,val,accn,fp,form,filed,frame,start,monthWindow,yearMonthDay
0,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-07-31,9000000.00,0001045810-16-000300,Q2,10-Q,2016-08-23,CY2016Q2,2016-05-02,3.0,20160731
1,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-10-30,9000000.00,0001045810-16-000353,Q3,10-Q,2016-11-22,NaN,2016-02-01,9.0,20161030
2,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2015-10-25,21.63,0001045810-15-000173,Q3,10-Q,2015-11-18,NaN,2015-01-26,9.0,20151025
3,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-07-31,42.06,0001045810-16-000300,Q2,10-Q,2016-08-23,NaN,2016-02-01,6.0,20160731
4,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-10-30,42.06,0001045810-16-000353,Q3,10-Q,2016-11-22,NaN,2016-02-01,9.0,20161030


In [11]:
from source.scraper import get_StockPrices

df = get_StockPrices('AAPL', startSelect='1986-01-02', endSelect='2022-01-02')
df.head()#11

,adjClose,ticker,quarter_meanADJclose,month_meanADJclose,yearMonthDay
0,0.076573,AAPL,0.084571,0.079429,19860102
1,0.077003,AAPL,0.084571,0.079429,19860103
2,0.076573,AAPL,0.084571,0.079429,19860106
3,0.079155,AAPL,0.084571,0.079429,19860107
4,0.078725,AAPL,0.084571,0.079429,19860108


In [12]:
from source.scraper import generate_CIK_TICKER

df = generate_CIK_TICKER()
df.head()

,cik_str,ticker,title,cik10D
0,320193,AAPL,Apple Inc.,0000320193
1,789019,MSFT,MICROSOFT CORP,0000789019
2,1045810,NVDA,NVIDIA CORP,0001045810
3,1652044,GOOGL,Alphabet Inc.,0001652044
4,1018724,AMZN,AMAZON COM INC,0001018724


In [29]:
from spark_deltalake.spark_operations import sparkDelta
from pyspark.sql.functions import col 


spark = sparkDelta()

hadoop-azure is available
azure-storage is available


In [31]:
path = spark.blobPath + '/SEC_filings'
#SEC_filings stock_prices
spark.spark.sql(f'SELECT COUNT(DISTINCT ticker) FROM delta.`{path}`').show()

+----------------------+
|count(DISTINCT ticker)|
+----------------------+
|                    89|
+----------------------+



In [16]:
from source.scraper import get_SEC_filings

ticker = 'AAPL'
filings = get_SEC_filings('CIK0001467373', 'ACN')
filings.head()#35

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,fileURL,yearMonthDay,ticker
0,0001467373-24-000248,2024-08-16,2024-08-15,2024-08-16T16:17:17.000Z,,4,,,,4750,0,0,xslF345X05/wk-form4_1723839430.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
1,0001467373-24-000247,2024-08-16,2024-08-15,2024-08-16T16:17:06.000Z,,4,,,,5688,0,0,xslF345X05/wk-form4_1723839420.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
2,0001467373-24-000246,2024-08-16,2024-08-15,2024-08-16T16:16:55.000Z,,4,,,,4802,0,0,xslF345X05/wk-form4_1723839409.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
3,0001467373-24-000245,2024-08-16,2024-08-15,2024-08-16T16:16:45.000Z,,4,,,,4696,0,0,xslF345X05/wk-form4_1723839398.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
4,0001467373-24-000244,2024-08-16,2024-08-15,2024-08-16T16:16:33.000Z,,4,,,,5596,0,0,xslF345X05/wk-form4_1723839387.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN


In [17]:
pydf = spark.get_spark_dataframe('SEC_filings', True)
pydf.show()

+--------------------+----------+----------+--------------------+---+------+----------+----------+-----+-------+------+------------+--------------------+---------------------+--------------------+------------+------+
|     accessionNumber|filingDate|reportDate|  acceptanceDateTime|act|  form|fileNumber|filmNumber|items|   size|isXBRL|isInlineXBRL|     primaryDocument|primaryDocDescription|             fileURL|yearMonthDay|ticker|
+--------------------+----------+----------+--------------------+---+------+----------+----------+-----+-------+------+------------+--------------------+---------------------+--------------------+------------+------+
|0001918704-24-000129|2024-08-28|          |2024-08-28T14:46:...| 33| 424B2|333-268718| 241254361|     |1227497|     0|           0|       form424b2.htm|                424B2|https://www.sec.g...|    20240828|   BAC|
|0001918704-24-000127|2024-08-28|          |2024-08-28T14:42:...| 33| 424B2|333-268718| 241254349|     | 951708|     0|           0|

In [18]:
pydf = spark.get_spark_dataframe('company_facts', True)
pydf.show()

+--------------------+--------------------+--------------------+----------+-------------+--------------------+---+----+----------+---------+----------+-----------+------------+
|             finType|               label|         description|       end|          val|                accn| fp|form|     filed|    frame|     start|monthWindow|yearMonthDay|
+--------------------+--------------------+--------------------+----------+-------------+--------------------+---+----+----------+---------+----------+-----------+------------+
|CashAndCashEquiva...|Cash and Cash Equ...|Amount of increas...|2008-12-31|      -1.78E8|0000732717-10-000013| FY|10-K|2010-02-25|      NaN|2008-01-01|       12.0|    20081231|
|CashAndCashEquiva...|Cash and Cash Equ...|Amount of increas...|2009-12-31|       2.01E9|0000732717-10-000013| FY|10-K|2010-02-25|      NaN|2009-01-01|       12.0|    20091231|
|CashPaidForCapita...|Cash Paid for Cap...|The amount of cas...|2007-12-31|      -1.71E8|0000732717-10-000013| FY|1

In [19]:
ticker = 'AOS'
loadedTickers = spark.get_min_max_fillings_date_filter_ticker()
loadedTickers.head()

,ticker,max_time,min_time
0,A,2024-08-22,2013-12-31
1,AAL,2024-07-30,2013-12-09
2,AAPL,2024-08-26,2013-10-15
3,ABBV,2024-08-14,2014-12-31
4,ABNB,2024-08-20,2012-10-24


In [20]:
spark.sparkStop()

In [21]:
from source.scraper import get_spy500_formWiki

wiki = get_spy500_formWiki()
wiki.head()

,Symbol,Security,GICS_Sector,GICS_Sub_Industry,Headquarters_Location,Date_added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [22]:
"""
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm


https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports

https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

/0001140361|24|024352/
accessionNumber - 0001140361-24-024352

"""

'\nhttps://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm\n\n\nhttps://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports\n\nhttps://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json\n\n/0001140361|24|024352/\naccessionNumber - 0001140361-24-024352\n\n'